In [2]:
import numpy as np
import pylas
import random
import math
from scipy import stats
import numpy_indexed as npi
import os

In [2]:
## read point clouds
pcd = pylas.read("C:\\Users\\24494\\OneDrive\\RANSAC\\Assignment01-Data-everybuilding\\BM1.las")
print('statistic of pcd: max z={}, min z={}, diff={}'.format(max(pcd.z), min(pcd.z), (max(pcd.z)-min(pcd.z))))

statistic of pcd: max z=107.403, min z=103.48, diff=3.923000000000002


In [13]:
#### change height
# pcd_withheight = pcd.points
# find mode for this dataset
pcd_z_mode = stats.mode(pcd.z)[0][0]
print('z value max={}, min={}, mode={}'.format(max(pcd.z),min(pcd.z), pcd_z_mode))
new_file = pylas.create(point_format_id=2, file_version="1.2")

z value max=110.142, min=103.492, mode=109.011


In [14]:
# choose points with z value more than the mode
new_file.points = pcd.points[pcd.z>(pcd_z_mode)]

# print(pcd.points)
# print(new_file.x)
#### save las file
new_file.write("C:\\Users\\24494\\Desktop\\code-assignment01\\Assignment01-Data\\BM1_CLEAN.las")

[(101069, 859010, 4166, 33280, 9, 0, 0, 0, 0, 34048, 34048, 27392)
 (101069, 859995, 6266, 27392, 9, 0, 0, 0, 0, 30976, 26880, 22784)
 (100740, 859995, 4872, 17664, 9, 0, 0, 0, 0, 20992, 16896, 13056) ...
 (109281, 874777, 8152, 18688, 9, 0, 0, 0, 0, 17920, 19200, 19200)
 (109281, 876091, 7765, 23296, 9, 0, 0, 0, 0, 22016, 24064, 23040)
 (109281, 875763, 7876, 23808, 9, 0, 0, 0, 0, 22528, 24832, 24064)]
[102.711 103.04  103.368 101.069 100.74  101.069 101.069 100.74  101.069
 100.74  101.069 100.74  102.054 101.726 101.397 102.054 101.726 101.397
 102.054 101.726 101.397 103.04  102.711 102.383 103.04  102.711 102.383
 103.04  102.711 102.383 102.054 101.726 101.397 102.054 101.726 101.397
 102.054 101.726 101.397 103.04  102.711 102.383 103.04  102.711 102.383
 103.04  102.711 102.383 104.025 103.697 103.368 104.025 103.697 103.368
 104.025 103.697 103.368 104.025 103.697 103.368 104.025 103.697 103.368
 104.025 103.697 103.368  98.769  99.755  99.426  99.755 101.069 100.74
 100.412 1

In [3]:
##Implementing RANSAC
best_interior = np.array([])
best_interior_idx = np.array([])
best_plane = []

class_number = 10

In [13]:
pcd = pylas.read("C:\\Users\\24494\\OneDrive\\RANSAC\\Assignment01-Data-everybuilding\\BM1.las")
point_count = pcd.header.point_count
pcd_p = np.array([pcd.x, pcd.y, pcd.z]) # 3* n_points
pcd_p = pcd_p.T   # n_points* 3
print('point clouds size: ',pcd_p.shape)
Max_Iter = 300
thres_dis = 0.30

point clouds size:  (2403, 3)


In [14]:
for i in range(Max_Iter):
    print('=======================iteration no. ', i, '=======================')
    # Step 1: add 3 random points
    random.seed()
    start_points_idx = [random.randint(0, point_count-1) for i_l in range(3)] # start_points = np.random.randint(0, self.point_count, size=3)
    print('start_points_idx: ', start_points_idx)
    p1 = [pcd.x[start_points_idx[0]], pcd.y[start_points_idx[0]], pcd.z[start_points_idx[0]]]
    p2 = [pcd.x[start_points_idx[1]], pcd.y[start_points_idx[1]], pcd.z[start_points_idx[1]]]
    p3 = [pcd.x[start_points_idx[2]], pcd.y[start_points_idx[2]], pcd.z[start_points_idx[2]]]
    print('3 random points:\np1: {}\np2: {}\np3: {}\n'.format(p1, p2, p3))
    # Step 2: Calculate plane contains p1, p2 and p3
    # Plane equation: ax + by + cz + d = 0
    # p1: x1, y1, z1
    # p2: x2, y2, z2
    # p3: x3, y3, z3
    a = (p2[1] - p1[1]) * (p3[2] - p1[2]) - (p2[2] - p1[2]) * (p3[1] - p1[1]) # a = (y2 - y1)*(z3 - z1) - (z2 - z1)*(y3 - y1)
    b = (p2[2] - p1[2]) * (p3[0] - p1[0]) - (p2[0] - p1[0]) * (p3[2] - p1[2]) # b = (z2 - z1)*(x3 - x1) - (x2 - x1)*(z3 - z1)
    c = (p2[0] - p1[0]) * (p3[1] - p1[1]) - (p2[1] - p1[1]) * (p3[0] - p1[0]) # c = (x2 - x1)*(y3 - y1) - (y2 - y1)*(x3 - x1)
    d = -(a * p1[0] + b * p1[1] + c * p1[2])# d = -(a*x1 + b*y1 + c*z1)
    plane_len = max(0.1, math.sqrt(a*a + b*b + c*c))

    # Step 3: add points into interior points
    # Skip points have be chosen as seed
    # pcd_p_search = np.delete(self.pcd_p, start_points_idx, axis=0)

    #  Calculate distance between the point and the plane
    #  (now all point are in calculating, including 3 samples)
    dist = np.abs((a*pcd_p[:, 0] + b * pcd_p[:, 1] + c * pcd_p[:, 2] +d)) / plane_len
    print(dist[0], dist[start_points_idx[0]], dist[start_points_idx[1]], dist[start_points_idx[2]])
    print('distance from every point to this plane: ', dist)

    # Add points in distance threshold as interior points
    interior_idx = np.where((dist <= thres_dis))[0]
#     print('interior max dist: {} : {}, min dist: {} : {}'.format(
#         max(dist), max(dist[interior_idx]), min(dist[interior_idx]), min(dist)))
#     print(interior_idx.shape)
    pcd_p_interior = pcd_p[interior_idx, :]

    # update the best model
    if (pcd_p_interior.shape[0] > best_interior.shape[0]) and (pcd_p_interior.shape[0]< point_count): # and (pcd_p_interior.shape[0]< self.point_count/2 * 1.15):
        print('updating...')
        best_interior = pcd_p_interior
        best_interior_idx = interior_idx
        best_plane = [a, b, c, d, plane_len]
        pcd.classification[interior_idx] = pcd.classification[interior_idx] + class_number

=======================iteration no.  0 =======================
start_points_idx:  [80, 366, 511]
3 random points:
p1: [3466694.763, 5469880.69, 105.111]
p2: [3466699.033, 5469877.077, 105.94800000000001]
p3: [3466697.062, 5469879.376, 105.382]

1.7768577644791093 0.0 0.0 0.0
distance from every point to this plane:  [1.77685776 2.26053023 0.96814917 ... 2.65135843 2.98217312 2.95172478]
=======================iteration no.  1 =======================
start_points_idx:  [2022, 396, 831]
3 random points:
p1: [3466699.69, 5469890.545, 105.167]
p2: [3466699.69, 5469878.391, 106.385]
p3: [3466701.99, 5469881.676, 107.236]

0.051016403132418675 0.0 2.361944468308453e-10 2.361944468308453e-10
distance from every point to this plane:  [0.0510164  0.34145052 0.63573396 ... 2.66245446 2.19470561 2.06061877]
=======================iteration no.  2 =======================
start_points_idx:  [538, 1552, 1961]
3 random points:
p1: [3466698.048, 5469880.362, 106.263]
p2: [3466704.618, 5469884.304, 10

3 random points:
p1: [3466696.405, 5469876.748, 104.68]
p2: [3466705.275, 5469878.391, 105.449]
p3: [3466707.903, 5469881.676, 105.552]

0.34212753677418556 0.0 1.8659006918031512e-11 3.7318013836063024e-11
distance from every point to this plane:  [0.34212754 0.76444283 0.25955912 ... 0.89847462 0.51036289 0.49629304]
=======================iteration no.  40 =======================
start_points_idx:  [735, 2167, 180]
3 random points:
p1: [3466697.391, 5469882.333, 106.359]
p2: [3466700.019, 5469892.516, 103.803]
p3: [3466697.062, 5469875.763, 105.201]

2.400048230870169 0.0 0.0 4.6524631229333475e-10
distance from every point to this plane:  [2.40004823 2.53840789 2.27587875 ... 2.66836612 2.53071843 2.23059883]
=======================iteration no.  41 =======================
start_points_idx:  [1685, 389, 463]
3 random points:
p1: [3466702.318, 5469885.946, 106.826]
p2: [3466698.376, 5469878.062, 105.55799999999999]
p3: [3466703.304, 5469877.734, 105.456]

1.2813928063756288 0.0 0.0 

=======================iteration no.  82 =======================
start_points_idx:  [762, 288, 502]
3 random points:
p1: [3466699.69, 5469880.69, 106.65]
p2: [3466703.304, 5469875.763, 104.787]
p3: [3466695.42, 5469879.705, 105.341]

0.07774928139209498 0.0 0.0 0.0
distance from every point to this plane:  [0.07774928 0.13339054 0.1237739  ... 9.3090524  8.94022295 8.90118497]
=======================iteration no.  83 =======================
start_points_idx:  [995, 2311, 1300]
3 random points:
p1: [3466708.231, 5469880.362, 105.473]
p2: [3466706.26, 5469885.289, 105.396]
p3: [3466692.135, 5469886.932, 105.083]

1.031405585406476 0.0 1.4031798101286141e-11 7.015899050643071e-12
distance from every point to this plane:  [1.03140559 1.44362813 0.46355476 ... 1.61461471 1.24868824 1.2586881 ]
updating...
=======================iteration no.  84 =======================
start_points_idx:  [1673, 306, 1277]
3 random points:
p1: [3466701.333, 5469886.275, 106.85]
p2: [3466704.618, 5469875.763,

3 random points:
p1: [3466702.318, 5469891.202, 105.695]
p2: [3466705.932, 5469878.719, 105.379]
p3: [3466703.961, 5469877.405, 105.178]

0.36203533185555437 0.0 6.32299108056721e-11 6.32299108056721e-11
distance from every point to this plane:  [0.36203533 0.75554388 0.15179467 ... 2.23816841 1.85706922 1.8506945 ]
=======================iteration no.  119 =======================
start_points_idx:  [1967, 1989, 908]
3 random points:
p1: [3466696.405, 5469889.888, 104.842]
p2: [3466699.033, 5469888.902, 105.621]
p3: [3466705.932, 5469878.062, 105.294]

3.0988918154619087 0.0 0.0 3.0779141056609734e-10
distance from every point to this plane:  [3.09889182 2.81606605 3.34754059 ... 4.66154253 4.22078553 4.11722457]
=======================iteration no.  120 =======================
start_points_idx:  [1468, 2310, 1681]
3 random points:
p1: [3466701.333, 5469882.99, 107.23]
p2: [3466705.932, 5469885.289, 105.517]
p3: [3466701.661, 5469886.603, 106.685]

5.381769477400744 0.0 2.2272679883092

distance from every point to this plane:  [0.87525546 0.6123726  1.04396312 ... 6.63479916 6.25979963 6.23295744]
=======================iteration no.  150 =======================
start_points_idx:  [507, 2092, 2202]
3 random points:
p1: [3466696.077, 5469880.033, 105.598]
p2: [3466703.304, 5469890.545, 106.096]
p3: [3466702.318, 5469892.516, 104.766]

1.4590322836912966 0.0 1.7725891902261162e-10 1.0789673331811142e-10
distance from every point to this plane:  [1.45903228 1.89204009 0.70436503 ... 0.897612   0.44222972 0.29362813]
=======================iteration no.  151 =======================
start_points_idx:  [866, 743, 2267]
3 random points:
p1: [3466704.289, 5469882.333, 106.466]
p2: [3466699.033, 5469881.347, 106.475]
p3: [3466706.917, 5469883.318, 105.588]

3.868299292987504 0.0 0.0 0.0
distance from every point to this plane:  [3.86829929 4.34898155 2.74859341 ... 8.96595469 9.1926581  9.23706813]
=======================iteration no.  152 =======================
start_points

3 random points:
p1: [3466696.734, 5469874.12, 103.947]
p2: [3466695.42, 5469889.559, 104.802]
p3: [3466706.26, 5469882.99, 105.965]

0.2520667848070248 0.0 1.8530252010798913e-10 1.8530252010798913e-10
distance from every point to this plane:  [0.25206678 0.13055245 0.73954024 ... 2.5962379  2.19371365 2.16241816]
=======================iteration no.  188 =======================
start_points_idx:  [0, 2341, 817]
3 random points:
p1: [3466695.092, 5469877.405, 104.182]
p2: [3466706.917, 5469886.275, 104.694]
p3: [3466700.676, 5469882.004, 107.337]

0.0 0.0 0.0 3.448472496764829e-10
distance from every point to this plane:  [0.         0.21544237 0.72305817 ... 8.54424533 8.6990688  8.89552838]
=======================iteration no.  189 =======================
start_points_idx:  [531, 12, 1980]
3 random points:
p1: [3466696.405, 5469880.362, 105.633]
p2: [3466694.106, 5469879.048, 104.881]
p3: [3466699.362, 5469889.559, 105.384]

1.257606155956331 0.0 9.842507476741763e-11 0.0
distance f

3 random points:
p1: [3466705.603, 5469882.661, 106.328]
p2: [3466702.647, 5469882.99, 106.876]
p3: [3466699.033, 5469884.304, 107.311]

5.452793538020606 0.0 0.0 0.0
distance from every point to this plane:  [5.45279354e+00 5.92170941e+00 4.67990194e+00 ... 1.91625738e-01
 8.19967863e-02 7.00333737e-04]
=======================iteration no.  225 =======================
start_points_idx:  [282, 1928, 2347]
3 random points:
p1: [3466703.304, 5469876.42, 105.089]
p2: [3466695.42, 5469888.902, 104.934]
p3: [3466705.932, 5469886.932, 105.125]

0.7582589864688253 0.0 0.0 8.04959645276733e-12
distance from every point to this plane:  [0.75825899 1.16854274 0.19629723 ... 1.41483112 1.04997823 1.0610824 ]
=======================iteration no.  226 =======================
start_points_idx:  [151, 833, 970]
3 random points:
p1: [3466696.077, 5469875.434, 105.034]
p2: [3466703.304, 5469881.347, 106.84]
p3: [3466707.246, 5469878.719, 105.25]

1.6643756388578717 0.0 0.0 0.0
distance from every point

3 random points:
p1: [3466703.304, 5469879.376, 105.544]
p2: [3466697.391, 5469884.304, 106.749]
p3: [3466697.062, 5469891.202, 103.493]

5.8263695894964345 0.0 0.0 0.0
distance from every point to this plane:  [5.82636959 6.27523217 5.00164721 ... 4.31780815 4.43842773 4.2657871 ]
=======================iteration no.  257 =======================
start_points_idx:  [154, 1531, 656]
3 random points:
p1: [3466695.749, 5469876.42, 103.532]
p2: [3466703.632, 5469884.632, 106.406]
p3: [3466703.961, 5469880.033, 105.829]

0.6331386729721293 0.0 0.0 0.0
distance from every point to this plane:  [0.63313867 0.28104696 1.04156667 ... 3.89424128 3.47742125 3.42476987]
=======================iteration no.  258 =======================
start_points_idx:  [2400, 1795, 347]
3 random points:
p1: [3466703.304, 5469894.158, 103.655]
p2: [3466697.391, 5469887.917, 105.671]
p3: [3466696.734, 5469878.062, 105.283]

1.599330210512373 0.0 1.269476201930366e-10 2.538952403860732e-10
distance from every point 

3 random points:
p1: [3466690.821, 5469886.603, 104.754]
p2: [3466697.062, 5469888.902, 105.272]
p3: [3466697.719, 5469880.69, 106.06700000000001]

1.8506349884370603 0.0 3.4938871584441016e-11 3.4938871584441016e-11
distance from every point to this plane:  [1.85063499 2.2848375  1.20995212 ... 1.85676236 1.4639354  1.44334337]
=======================iteration no.  291 =======================
start_points_idx:  [1893, 1404, 1721]
3 random points:
p1: [3466702.975, 5469888.574, 105.926]
p2: [3466697.719, 5469883.647, 106.724]
p3: [3466704.289, 5469885.946, 105.985]

3.174326309007781 0.0 9.125074396319425e-11 1.825014879263885e-10
distance from every point to this plane:  [3.17432631 3.60363692 2.55114698 ... 1.85623882 1.52856247 1.57448519]
=======================iteration no.  292 =======================
start_points_idx:  [2104, 2103, 2133]
3 random points:
p1: [3466704.289, 5469890.545, 104.903]
p2: [3466703.632, 5469890.873, 105.813]
p3: [3466698.048, 5469891.859, 103.517]

14.63

In [15]:
new_file = pylas.create_from_header(pcd.header)
new_file.points = pcd.points[best_interior_idx]
new_file.write(r"C:/Users/24494/OneDrive/RANSAC/BM10.las".format(i))

In [16]:
pcd.points = np.delete(pcd.points, best_interior_idx)
print(pcd.header.point_count)
i += 1

1679


In [17]:
"EVALUATE"

'EVALUATE'

In [18]:
gt_path = 'C:/Users/24494/OneDrive/RANSAC/Assignment01-Data-everybuilding/gt/BM1/'
pr_path = 'C:/Users/24494/OneDrive/RANSAC/'
filename  = 'BM10.las'

In [19]:
gt_i = os.path.join(gt_path, filename)
pr_i = os.path.join(pr_path, filename)

# read data
pcd_gt = pylas.read(gt_i) # Data01-cleanheight-1-roofblock.las: pcd.header.point_count =72590
pcd_pr = pylas.read(pr_i)
print('statistic of pcd_gt: max z={}, min z={}, diff={}'.format(max(pcd_gt.z), min(pcd_gt.z), (max(pcd_gt.z)-min(pcd_gt.z))))

statistic of pcd_gt: max z=107.403, min z=104.184, diff=3.2190000000000083


In [20]:
# arrange data
pcd_gt_p = np.array([pcd_gt.x, pcd_gt.y, pcd_gt.z]) # 3* n_points, gt
pcd_gt_p = pcd_gt_p.T   # n_points* 3
pcd_pr_p = np.array([pcd_pr.x, pcd_pr.y, pcd_pr.z]) # 3* n_points, predict result
pcd_pr_p = pcd_pr_p.T   # n_points* 3

In [21]:
# find intersection
intersection = npi.intersection(pcd_gt_p, pcd_pr_p)

# calculate accuracy
acc = intersection.shape[0] / pcd_gt_p.shape[0]

print('acca of {} is:   {}'.format(gt_i, acc))

acca of C:/Users/24494/OneDrive/RANSAC/Assignment01-Data-everybuilding/gt/BM1/BM10.las is:   0.8878248974008208
